## 장애인 관련 법률

- 장애인 복지법
- 장애인 노인 임산부 등의 편의증진보장에 관한 법률
- 사회복지사업법
- 사회복지법인 및 사회복지시설 재무회계 규칙
- 장애인고용촉진 및 직업재활법
- 장애인 등에 대한 특수교육법
- 장애인차별금지 및 권리구제 등에 관한 법률
- 장애인연금법

In [1]:
law_list = [
    '장애인 복지법',
    '장애인 노인 임산부 등의 편의증진보장에 관한 법률',
    '사회복지사업법',
    '사회복지법인 및 사회복지시설 재무회계 규칙',
    '장애인고용촉진 및 직업재활법',
    '장애인 등에 대한 특수교육법',
    '장애인차별금지 및 권리구제 등에 관한 법률',
    '장애인연금법',
]

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from datetime import datetime
import time
import os

def check_side_center_exists(driver):
    """#sideCenter 요소 존재 여부 확인"""
    try:
        side_center = driver.find_elements(By.XPATH, '//*[@id="sideCenter"]')
        return len(side_center) > 0
    except:
        return False

def parse_date(date_str):
    """'YYYY. M. D.' 형식의 날짜 문자열을 datetime 객체로 변환"""
    try:
        cleaned_date = date_str.replace(' ', '').replace('.', '')
        if len(cleaned_date) == 8:  # YYYYMMDD
            return datetime.strptime(cleaned_date, '%Y%m%d')
        return None
    except:
        return None

def get_latest_valid_row(driver, wait):
    """테이블에서 현재 날짜 이전의 가장 최근 법령 행의 링크를 찾는 함수"""
    try:
        table = wait.until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="viewHeightDiv"]/table/tbody'))
        )
        rows = table.find_elements(By.TAG_NAME, "tr")
        
        current_date = datetime.now()
        valid_rows = []
        
        for idx, row in enumerate(rows, 1):
            try:
                date_cell = row.find_element(By.XPATH, "./td[3]")
                date_text = date_cell.text.strip()
                enforcement_date = parse_date(date_text)
                
                if enforcement_date and enforcement_date <= current_date:
                    valid_rows.append((enforcement_date, idx))
            except:
                continue
        
        if valid_rows:
            valid_rows.sort(key=lambda x: x[0], reverse=True)
            row_idx = valid_rows[0][1]
            
            link_xpath = f'//*[@id="viewHeightDiv"]/table/tbody/tr[{row_idx}]/td[2]/a'
            title_link = driver.find_element(By.XPATH, link_xpath)
            
            return title_link
        
        return None
    except Exception as e:
        print(f"테이블 검색 중 오류 발생: {str(e)}")
        return None

def save_law_as_pdf(driver, wait):
    """현재 페이지의 법령을 PDF로 저장"""
    try:
        save_button = wait.until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="bdySaveBtn"]/img'))
        )
        driver.execute_script("arguments[0].click();", save_button)
        time.sleep(2)
        
        pdf_radio = wait.until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="FileSavePdf1"]'))
        )
        driver.execute_script("arguments[0].click();", pdf_radio)
        time.sleep(1)
        
        final_save_button = wait.until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="aBtnOutPutSave"]/img'))
        )
        driver.execute_script("arguments[0].click();", final_save_button)
        time.sleep(5)
        return True
    except Exception as e:
        print(f"PDF 저장 중 오류 발생: {str(e)}")
        return False

def search_and_save_laws_pdf(law_list, headless=False, download_path=None):
    """법령 검색 및 PDF 저장 메인 함수"""
    chrome_options = Options()
    chrome_options.add_argument('--start-maximized')
    
    if headless:
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--disable-gpu')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
    
    if download_path is None:
        download_path = os.path.join(os.path.expanduser("~"), "Desktop", "법령다운로드")
    
    if not os.path.exists(download_path):
        os.makedirs(download_path)
    
    prefs = {
        "download.default_directory": download_path,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    }
    chrome_options.add_experimental_option("prefs", prefs)
    
    driver = webdriver.Chrome(options=chrome_options)
    wait = WebDriverWait(driver, 10)
    
    try:
        for law in law_list:
            try:
                print(f"\n{law} 처리 중...")
                
                driver.get("https://www.law.go.kr/LSW/main.html")
                
                search_box = wait.until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="query"]'))
                )
                search_box.clear()
                search_box.send_keys(law)
                
                search_button = wait.until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="inner"]/a'))
                )
                driver.execute_script("arguments[0].click();", search_button)
                time.sleep(3)
                
                if check_side_center_exists(driver):
                    print(f"{law}: 직접 법령 페이지 - PDF 저장 시작")
                    save_law_as_pdf(driver, wait)
                else:
                    print(f"{law}: 검색 결과 페이지 - 현재 유효한 법령 검색 중")
                    title_link = get_latest_valid_row(driver, wait)
                    if title_link:
                        try:
                            print(f"선택된 법령: {title_link.text}")
                            driver.execute_script("arguments[0].click();", title_link)
                            time.sleep(3)
                            save_law_as_pdf(driver, wait)
                        except Exception as e:
                            print(f"법령 클릭 중 오류: {str(e)}")
                    else:
                        print(f"{law}: 현재 유효한 법령을 찾을 수 없습니다.")
                
            except Exception as e:
                print(f"{law} 처리 중 오류 발생: {str(e)}")
                continue
            
    except Exception as e:
        print(f"전체 작업 중 오류 발생: {str(e)}")
    
    finally:
        driver.quit()

if __name__ == "__main__":
    law_list = [
        '장애인 복지법',
        '장애인 노인 임산부 등의 편의증진보장에 관한 법률',
        '사회복지사업법',
        '사회복지법인 및 사회복지시설 재무회계 규칙',
        '장애인고용촉진 및 직업재활법',
        '장애인 등에 대한 특수교육법',
        '장애인차별금지 및 권리구제 등에 관한 법률',
        '장애인연금법',
    ]
    
    search_and_save_laws_pdf(law_list, download_path="./files")
    # search_and_save_laws_pdf(law_list, headless=True)
    # search_and_save_laws_pdf(law_list)


장애인 복지법 처리 중...
장애인 복지법: 직접 법령 페이지 - PDF 저장 시작

장애인 노인 임산부 등의 편의증진보장에 관한 법률 처리 중...
장애인 노인 임산부 등의 편의증진보장에 관한 법률: 검색 결과 페이지 - 현재 유효한 법령 검색 중
선택된 법령: 장애인ㆍ노인ㆍ임산부 등의 편의증진 보장에 관한 법률 시행규칙

사회복지사업법 처리 중...
사회복지사업법: 직접 법령 페이지 - PDF 저장 시작

사회복지법인 및 사회복지시설 재무회계 규칙 처리 중...
사회복지법인 및 사회복지시설 재무회계 규칙: 검색 결과 페이지 - 현재 유효한 법령 검색 중
선택된 법령: 사회복지법인 및 사회복지시설 재무ㆍ회계 규칙

장애인고용촉진 및 직업재활법 처리 중...
장애인고용촉진 및 직업재활법: 직접 법령 페이지 - PDF 저장 시작

장애인 등에 대한 특수교육법 처리 중...
장애인 등에 대한 특수교육법: 직접 법령 페이지 - PDF 저장 시작

장애인차별금지 및 권리구제 등에 관한 법률 처리 중...
장애인차별금지 및 권리구제 등에 관한 법률: 직접 법령 페이지 - PDF 저장 시작

장애인연금법 처리 중...
장애인연금법: 직접 법령 페이지 - PDF 저장 시작
